In [39]:
# This is a new notebook for the NLP stuff. Feel free to rip some of this for the numeric stuff.
##
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import string
import re
import unidecode

In [3]:
# Read in the restaurant data
restaurants = pd.read_json('restaurants.json').T

In [4]:
restaurants.shape

(14089, 16)

In [5]:
restaurants.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
le-pain-quotidien-new-york-137,ce605OMECZ3jLZtk6dCDHA,le-pain-quotidien-new-york-137,Le Pain Quotidien,https://s3-media3.fl.yelpcdn.com/bphoto/z-YqDX...,False,https://www.yelp.com/biz/le-pain-quotidien-new...,84,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.0,"{'latitude': 40.7112998613948, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '395 S End Ave', 'address2': '', ...",+16466188930,(646) 618-8930,812.911847
creperie-new-york-6,B8yFj4qt-HgMCbF5S-GkgA,creperie-new-york-6,Creperie,https://s3-media1.fl.yelpcdn.com/bphoto/uydGkc...,False,https://www.yelp.com/biz/creperie-new-york-6?a...,19,"[{'alias': 'creperies', 'title': 'Creperies'}]",2.5,"{'latitude': 40.711547382455, 'longitude': -74...",[],$,"{'address1': '255 Liberty St', 'address2': '',...",,,762.969495
dos-toros-taqueria-new-york-5,WZLhPYaYSFy7M_-Jh1VuNw,dos-toros-taqueria-new-york-5,Dos Toros Taqueria,https://s3-media1.fl.yelpcdn.com/bphoto/fNdfBJ...,False,https://www.yelp.com/biz/dos-toros-taqueria-ne...,132,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,"{'latitude': 40.712342, 'longitude': -74.01528}",[],$$,"{'address1': '225 Liberty St', 'address2': 'St...",+13472189518,(347) 218-9518,756.603792
umami-burger-new-york-17,tFQRwk9SVOFgpMBZ55UrLw,umami-burger-new-york-17,Umami Burger,https://s3-media2.fl.yelpcdn.com/bphoto/uvquI3...,False,https://www.yelp.com/biz/umami-burger-new-york...,333,"[{'alias': 'burgers', 'title': 'Burgers'}]",3.0,"{'latitude': 40.712344, 'longitude': -74.015511}","[delivery, pickup]",$$,"{'address1': '225 Liberty St', 'address2': 'St...",+19177284400,(917) 728-4400,756.603792
naya-brookfield-place-new-york-3,jhs35REmv8Yo9-Z27KQJSA,naya-brookfield-place-new-york-3,NAYA - Brookfield Place,https://s3-media3.fl.yelpcdn.com/bphoto/Zd6LgX...,False,https://www.yelp.com/biz/naya-brookfield-place...,32,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.5,"{'latitude': 40.71156, 'longitude': -74.01533}","[pickup, delivery]",NaN,"{'address1': '225 Liberty St', 'address2': 'Fl...",+19299468404,(929) 946-8404,756.603792


In [6]:
# Read in the review data
# Define dataframe for reviews
reviews = pd.DataFrame()

# Loop through all review .jsons
for letter in string.ascii_lowercase + '0':
    path = "{}/reviews/{}.json".format(os.getcwd(), letter)
    with open(path, "r") as json_file:
        alias_to_reviews = json.loads(json_file.read())

        # For each alias, get reviews
        for alias in alias_to_reviews.keys():
            # Flatten structure into dataframe, track alias for easy groupby/merge with restaurants
            review = pd.json_normalize(alias_to_reviews[alias], sep="_")
            review['alias'] = alias
            
            # Stitch into final dataframe
            reviews = pd.concat([reviews, review], ignore_index=True)
reviews.shape

(331693, 8)

In [7]:
reviews.head()

,photoCount,reviewCount,eliteYear,localizedDate,rating,comment_text,comment_language,alias
0,190.0,1421.0,NaN,8/6/2023,3.0,Overall the food is pretty good but portions c...,en,ani-ramen-house-new-york
1,580.0,148.0,2023.0,6/8/2023,4.0,We&#39;ve been to ani ramen in Montclair New J...,en,ani-ramen-house-new-york
2,531.0,186.0,2023.0,5/24/2023,4.0,The real deal Japanese rice. Crispy tofu was t...,en,ani-ramen-house-new-york
3,770.0,213.0,2023.0,7/18/2023,3.0,This is located within Brookfield place. You c...,en,ani-ramen-house-new-york
4,692.0,185.0,2023.0,11/12/2022,1.0,I have been wanting to try Ani Ramen for the l...,en,ani-ramen-house-new-york


In [8]:
# Find restaurants with missing reviews
missing_reviews = restaurants['name'][restaurants['alias'].isin(reviews['alias']) == False].reset_index()
missing_reviews.columns = ['alias', 'name']
missing_reviews

,alias,name
0,sauce-pizzeria-new-york-8,Sauce Pizzeria
1,sams-crispy-chicken-new-york-4,Sam's Crispy Chicken
2,springbone-kitchen-new-york-7,Springbone Kitchen
3,atrio-wine-bar-and-restaurant-new-york-2,Atrio Wine Bar & Restaurant
4,er-hotpot-new-york-4,ER Hotpot
...,...,...
3207,edgewater-sandwich-edgewater,Edgewater Sandwich
3208,india-grill-and-curry-edgewater,India Grill And Curry
3209,plaza-yankees-restaurant-bronx,Plaza Yankees Restaurant
3210,super-m-deli-new-york,Super M Deli


In [9]:
missing_reviews.nunique() # Multiple aliases for restaurants, many to one relationship due to locations I assume

alias    3212
name     3021
dtype: int64

In [10]:
missing_reviews['name'].value_counts()

Tyga Bites                  16
Farmer's Fridge             11
Market Plate                11
Dunkin'                     10
Kennedy Fried Chicken        8
                            ..
Family diner                 1
I & S Gourmet Deli           1
The Hungry Vegan             1
Criollas Baked Empanadas     1
Justin’s Pizza               1
Name: name, Length: 3021, dtype: int64

In [11]:
missing_reviews[missing_reviews['name'] == "Tyga Bites"] # I mean not a biggie, just playing around
# Perhaps multiple locations (NY, Bronx, Brooklyn) could be an issue, may be better to analyze aliases and just keep a dictionary of this

,alias,name
168,tyga-bites-new-york-13,Tyga Bites
205,tyga-bites-new-york-20,Tyga Bites
1113,tyga-bites-new-york-19,Tyga Bites
1254,tyga-bites-new-york-17,Tyga Bites
1373,tyga-bites-new-york-53,Tyga Bites
1427,tyga-bites-new-york-3,Tyga Bites
1503,tyga-bites-new-york-23,Tyga Bites
1797,tyga-bites-new-york-22,Tyga Bites
1841,tyga-bites-the-bronx-5,Tyga Bites
2137,tyga-bites-new-york-51,Tyga Bites


In [12]:
# Let's see if any of the missing review data comes from the michelin
michelin = [
"Eleven Madison park",
"Le Bernardin",
"Masa",
"Per Se",
"Al Coro",
"Aquavit",
"Aska",
"Atera",
"Atomix",
"Blue Hill at Stone Barns",
"Daniel",
"Gabriel Kreuther",
"Jean-Georges",
"Jungsik",
"The Modern",
"Odo",
"Saga",
"Sushi Noz",
"Sixty Three Clinton",
"Bōm",
"Casa Mono",
"Caviar Russe",
"Clover Hill",
"Cote",
"Crown Shy",
"Dirt Candy",
"Essential by Christophe",
"Estela",
"Family Hill at Blue Hill",
"The Four Horseman",
"Francie",
"Frevo",
"Gramercy Tavern",
"Hirohisa",
"Icca",
"Jeju Noodle Bar",
"Jōji",
"Joomak Banjum",
"Jua",
"Kochi",
"Kosaka",
"L'Abeille",
"Le Coucou",
"Le Jardinier",
"Le Pavillon",
"Mari",
"Meju",
"The Musket Room",
"Noda",
"Noz 17",
"Oiji Mi",
"One White Street",
"Oxalis",
"Oxomoco",
"Red paper Clip",
"Restaurant Yuu",
"Rezdôra",
"Semma",
"Shion 69 Leonard Street",
"Shmoné",
"Sushi Amane",
"Sushi Ichimura",
"Sushi Nakazawa",
"Sushi Yasuda",
"Tempura Matsui",
"Torien",
"Torrisi",
"Tsukimi",
"Tuome",
"Vestry",
"Yoshino"
]

In [13]:
# Convert to ascii lowercase
mich = pd.Series([x.lower() for x in michelin])
review_res = pd.Series([x.lower() for x in missing_reviews['name']])
res = pd.Series([x.lower() for x in restaurants['name']])

missing_mich = mich.isin(review_res)
len(missing_mich), sum(missing_mich) # Well no Michelins show up as missing, but still feeling sus

(71, 0)

In [14]:
# Let's see if we can find the Michelins in our restaurant data...
missing_mich_res = mich.isin(res)
len(missing_mich_res), sum(missing_mich_res) # Well, we retain 57 Michelins, what about the other 14??

(71, 57)

In [15]:
# Let's find the 14, hard to systemitize, need more thought on this part - we don't have their aliases, so I'm just matching names
find_mich = mich[mich.isin(res) == False].reset_index()
find_mich # Example - aska has alias aska-brooklyn-3, didn't show up and I could not find 'Aska' in the .json - something to look out for. Others could just be name variation (i.e. probably just blue hill on Yelp, not the fancy)

,index,0
0,6,aska
1,9,blue hill at stone barns
2,22,clover hill
3,23,cote
4,28,family hill at blue hill
5,29,the four horseman
6,30,francie
7,36,jōji
8,41,l'abeille
9,46,meju


In [16]:
# Let's do a merger with restaurant name and reviews for analysis
# Fancy indexing Jeremy, let me cook
names = restaurants[['alias', 'name', 'categories', 'price']]
names

,alias,name,categories,price
le-pain-quotidien-new-york-137,le-pain-quotidien-new-york-137,Le Pain Quotidien,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",$$
creperie-new-york-6,creperie-new-york-6,Creperie,"[{'alias': 'creperies', 'title': 'Creperies'}]",$
dos-toros-taqueria-new-york-5,dos-toros-taqueria-new-york-5,Dos Toros Taqueria,"[{'alias': 'mexican', 'title': 'Mexican'}]",$$
umami-burger-new-york-17,umami-burger-new-york-17,Umami Burger,"[{'alias': 'burgers', 'title': 'Burgers'}]",$$
naya-brookfield-place-new-york-3,naya-brookfield-place-new-york-3,NAYA - Brookfield Place,"[{'alias': 'mediterranean', 'title': 'Mediterr...",NaN
...,...,...,...,...
super-m-deli-new-york,super-m-deli-new-york,Super M Deli,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",NaN
happy-wok-bronx,happy-wok-bronx,Happy Wok,"[{'alias': 'chinese', 'title': 'Chinese'}]",NaN
justins-pizza-bronx,justins-pizza-bronx,Justin's Pizza,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",$$
justin-s-pizza-bronx-2,justin-s-pizza-bronx-2,Justin’s Pizza,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",NaN


In [17]:
# Merge dataframes for analysis
df = pd.merge(reviews, names, on='alias')
df.shape

(331693, 11)

In [18]:
df.head()

,photoCount,reviewCount,eliteYear,localizedDate,rating,comment_text,comment_language,alias,name,categories,price
0,190.0,1421.0,NaN,8/6/2023,3.0,Overall the food is pretty good but portions c...,en,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN
1,580.0,148.0,2023.0,6/8/2023,4.0,We&#39;ve been to ani ramen in Montclair New J...,en,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN
2,531.0,186.0,2023.0,5/24/2023,4.0,The real deal Japanese rice. Crispy tofu was t...,en,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN
3,770.0,213.0,2023.0,7/18/2023,3.0,This is located within Brookfield place. You c...,en,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN
4,692.0,185.0,2023.0,11/12/2022,1.0,I have been wanting to try Ani Ramen for the l...,en,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN


In [19]:
# Trim what we will not use for efficiency
df.drop(['photoCount', 'comment_language'], axis=1, inplace=True)
df.head()

,reviewCount,eliteYear,localizedDate,rating,comment_text,alias,name,categories,price
0,1421.0,NaN,8/6/2023,3.0,Overall the food is pretty good but portions c...,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN
1,148.0,2023.0,6/8/2023,4.0,We&#39;ve been to ani ramen in Montclair New J...,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN
2,186.0,2023.0,5/24/2023,4.0,The real deal Japanese rice. Crispy tofu was t...,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN
3,213.0,2023.0,7/18/2023,3.0,This is located within Brookfield place. You c...,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN
4,185.0,2023.0,11/12/2022,1.0,I have been wanting to try Ani Ramen for the l...,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN


In [20]:
# Flatten the categories to just reflect the title, then we can check if it includes a specific tag for filtering (using .isin())
df['categories'] = df['categories'].apply(lambda x: [item['title'] for item in x])

# Rename columns for ease
df.columns = ['user_review_count', 'elite_year', 'review_date','rating','review','alias','name','categories','price']

df.head()

,user_review_count,elite_year,review_date,rating,review,alias,name,categories,price
0,1421.0,NaN,8/6/2023,3.0,Overall the food is pretty good but portions c...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN
1,148.0,2023.0,6/8/2023,4.0,We&#39;ve been to ani ramen in Montclair New J...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN
2,186.0,2023.0,5/24/2023,4.0,The real deal Japanese rice. Crispy tofu was t...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN
3,213.0,2023.0,7/18/2023,3.0,This is located within Brookfield place. You c...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN
4,185.0,2023.0,11/12/2022,1.0,I have been wanting to try Ani Ramen for the l...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN


In [21]:
# Start some processing of the text
dup_reviews = df[df['review'].duplicated(keep=False)]
dup_reviews # Hmm, so aliases may reflect the same location?? See the 2-bros-pizza example...

# Doesn't look like it here but if folks copy and paste same reviews for different restaurants, first of all, HA, and second we don't want them anyway

,user_review_count,elite_year,review_date,rating,review,alias,name,categories,price
3636,482.0,NaN,9/27/2022,5.0,There is a very kind and patient employee by t...,angelika-film-center-and-café-new-york-new-york-3,Angelika Film Center & Café - New York,"[Cinema, Cafes, Bars]",$$
4933,182.0,NaN,4/27/2022,4.0,Bottom line up front: check the store name and...,amarone-scarlatto-new-york,Amarone Scarlatto,[Italian],$$
4967,182.0,NaN,4/27/2022,4.0,Bottom line up front: check the store name and...,amarone-ny-new-york-2,Amarone NY,"[Italian, Seafood]",$$
5963,898.0,2023.0,7/30/2017,3.0,I was in and out because I knew exactly what I...,au-bon-pain-new-york-6,Au Bon Pain,[Sandwiches],$
6051,898.0,2023.0,9/16/2018,2.0,I was in and out because I knew exactly what I...,au-bon-pain-new-york-47,Au Bon Pain,"[Soup, Breakfast & Brunch, Sandwiches]",NaN
...,...,...,...,...,...,...,...,...,...
326715,5.0,NaN,11/14/2023,1.0,The bar is stinky like that old sour beer smel...,1803-new-york,1803 Nyc,"[Cajun/Creole, Seafood, Bars]",$$
326896,4.0,NaN,3/30/2018,1.0,"I ordered 4 pizzas for my office, and they cam...",5-boroughs-pizza-new-york-2,5 Boroughs Pizza,[Pizza],$
327153,274.0,2023.0,7/24/2022,3.0,"pizza itself was delicious, but the signs were...",2-bros-pizza-new-york-4,2 Bros Pizza,[Pizza],$
328269,274.0,2023.0,7/24/2022,3.0,"pizza itself was delicious, but the signs were...",2-bros-pizza-new-york-6,2 Bros Pizza,[Pizza],$


In [22]:
# Drop duplicate reviews
df = df[df['review'].duplicated() == False]
df.shape

(331501, 9)

In [23]:
# Get the michelin tag
review_res = pd.Series([x.lower() for x in missing_reviews['name']])
df['michelin'] = pd.Series([x.lower() for x in df['name']]).isin(mich)
df['michelin'].sum() / len(df), df['michelin'].sum() # So .07% is Michelin, we should probably sample for a classifier

(0.007858196506194551, 2605)

In [24]:
df.head()

,user_review_count,elite_year,review_date,rating,review,alias,name,categories,price,michelin
0,1421.0,NaN,8/6/2023,3.0,Overall the food is pretty good but portions c...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN,False
1,148.0,2023.0,6/8/2023,4.0,We&#39;ve been to ani ramen in Montclair New J...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN,False
2,186.0,2023.0,5/24/2023,4.0,The real deal Japanese rice. Crispy tofu was t...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN,False
3,213.0,2023.0,7/18/2023,3.0,This is located within Brookfield place. You c...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN,False
4,185.0,2023.0,11/12/2022,1.0,I have been wanting to try Ani Ramen for the l...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN,False


In [25]:
# Get list of unique chars to see what we may want to remove
set(' '.join(df['review']))

{'紅',
 '梨',
 '图',
 '古',
 '問',
 'ç',
 '.',
 '會',
 '作',
 'ぼ',
 '着',
 '搜',
 '巧',
 '過',
 '쪽',
 '错',
 '据',
 'Æ',
 '堡',
 '免',
 '＿',
 '盐',
 '瓜',
 '王',
 '译',
 '訳',
 '片',
 '夹',
 '芥',
 '햄',
 '豆',
 '翡',
 '辛',
 '鶏',
 'ل',
 '蔔',
 'k',
 '也',
 '握',
 '렁',
 '荞',
 '比',
 '能',
 '浓',
 '购',
 '어',
 '任',
 '尝',
 '腌',
 '＼',
 '又',
 '곳',
 '動',
 '烫',
 '水',
 'ツ',
 '次',
 '帷',
 '个',
 '收',
 '隆',
 'ن',
 '满',
 '酿',
 '壁',
 '洋',
 '星',
 '椰',
 'ñ',
 '凌',
 '프',
 '넣',
 '銀',
 '优',
 '飯',
 '都',
 '径',
 '萄',
 '번',
 '暴',
 '午',
 '昼',
 '黄',
 '血',
 '夠',
 'そ',
 '買',
 '础',
 '吝',
 '糍',
 '러',
 '構',
 '饅',
 '冻',
 '니',
 '意',
 '圆',
 '¼',
 '队',
 'ょ',
 '喝',
 '當',
 '順',
 '癮',
 '도',
 '乐',
 '別',
 '田',
 '됩',
 'V',
 '7',
 '西',
 '합',
 '棉',
 '。',
 '週',
 '빙',
 '豬',
 '么',
 '′',
 '服',
 '燒',
 '蓉',
 '提',
 '旨',
 '之',
 'â',
 '鍋',
 '确',
 '｜',
 'ỏ',
 '醒',
 '案',
 '打',
 '간',
 '氛',
 '去',
 '蜊',
 '약',
 '掉',
 'グ',
 'و',
 'o',
 '한',
 '링',
 '没',
 'E',
 '茄',
 '구',
 '蕎',
 '列',
 '賣',
 '난',
 '中',
 '春',
 '激',
 '怪',
 '藕',
 'q',
 '纯',
 '컵',
 '¯',
 '舞',
 '魚',
 '韓',
 '單',
 '嚟'

In [26]:
# Get the counts, let's see frequencies
char_count = df['review'].str.split('').explode().value_counts().to_frame().reset_index()
char_count.columns = ['char', 'count']

In [27]:
# They do say e is the most popular letter in the English language... my childhood fun fact is true
char_count

,char,count
0,,31771368
1,e,17193449
2,t,11893759
3,a,11574489
4,o,10080546
...,...,...
2391,蕎,1
2392,病,1
2393,铁,1
2394,玻,1


In [28]:
# Let's try some slices
char_count[char_count['count'] <= 80] # Just other chars it seems

,char,count
134,有,79
135,い,78
136,の,77
137,吃,77
138,た,72
...,...,...
2391,蕎,1
2392,病,1
2393,铁,1
2394,玻,1


In [29]:
df['review'] # See that &#39;... let's see the n-grams now and we can decide how to clean
# &#39; is the code for an apostrophe

0         Overall the food is pretty good but portions c...
1         We&#39;ve been to ani ramen in Montclair New J...
2         The real deal Japanese rice. Crispy tofu was t...
3         This is located within Brookfield place. You c...
4         I have been wanting to try Ani Ramen for the l...
                                ...                        
331688    I honestly think between 161-170th in the area...
331689    Awesome pizza for the area.  Prices can be a l...
331690    I called to order. When I gave my address, whi...
331691    I called this place to place a delivery order....
331692    Nonna&#39;s is quality pizza, and not just for...
Name: review, Length: 331501, dtype: object

In [30]:
from nltk import ngrams, FreqDist
from nltk.tokenize import TreebankWordTokenizer

In [31]:
# Tokenize the review data
tokens = df['review'].apply(TreebankWordTokenizer().tokenize)

# Generate n-grams for each row (try 2 words together)
n_grams = tokens.apply(lambda x: list(ngrams(x, 2)))

In [32]:
n_grams # See that sus index 1

0         [(Overall, the), (the, food), (food, is), (is,...
1         [(We, &), (&, #), (#, 39), (39, ;), (;, ve), (...
2         [(The, real), (real, deal), (deal, Japanese), ...
3         [(This, is), (is, located), (located, within),...
4         [(I, have), (have, been), (been, wanting), (wa...
                                ...                        
331688    [(I, honestly), (honestly, think), (think, bet...
331689    [(Awesome, pizza), (pizza, for), (for, the), (...
331690    [(I, called), (called, to), (to, order.), (ord...
331691    [(I, called), (called, this), (this, place), (...
331692    [(Nonna, &), (&, #), (#, 39), (39, ;), (;, s),...
Name: review, Length: 331501, dtype: object

In [33]:
# Flatten the list of n-grams
all_ngrams = [ngram for row in n_grams for ngram in row]

# Feed into NLTK Freq Dist
freq_dist = FreqDist(all_ngrams)

In [48]:
# Display the top N n-grams
print(freq_dist.most_common(10)) # We'll use this down the line when we start subsetting the data
# HTML patterns are still apparent, let's figure out how to parse this...
# Should've just looked at the JSON and saw the <br> LOL

[(('<', 'br'), 814781), (('br', '>'), 814781), (('&', '#'), 681411), (('#', '39'), 601819), (('39', ';'), 601818), (('>', '<'), 336433), ((';', 't'), 208469), ((';', 's'), 185562), ((',', 'and'), 140263), (('I', '&'), 119291)]


In [56]:
# Remove HTML tags from text using Beautiful Soup
# Why a library over regular expressions? Better edge case handling
from bs4 import BeautifulSoup
df['review'] = df['review'].apply(lambda x: unidecode.unidecode(BeautifulSoup(x, 'html.parser').get_text(separator=' '))) # The warning just means maybe not all contain HTML tags, it's fine. Unidecode to remove non-English chars

/var/folders/_q/gmdf_7c54956tq7fvnbprqw00000gn/T/ipykernel_30077/3999042358.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['review'] = df['review'].apply(lambda x: unidecode.unidecode(BeautifulSoup(x, 'html.parser').get_text(separator=' '))) # The warning just means maybe not all contain HTML tags, it's fine. Unidecode to remove non-English chars


In [57]:
df['review']

0         Overall the food is pretty good but portions c...
1         We've been to ani ramen in Montclair New Jerse...
2         The real deal Japanese rice. Crispy tofu was t...
3         This is located within Brookfield place. You c...
4         I have been wanting to try Ani Ramen for the l...
                                ...                        
331688    I honestly think between 161-170th in the area...
331689    Awesome pizza for the area.  Prices can be a l...
331690    I called to order. When I gave my address, whi...
331691    I called this place to place a delivery order....
331692    Nonna's is quality pizza, and not just for da ...
Name: review, Length: 331501, dtype: object

In [58]:
# Get the counts, let's see frequencies
char_count = df['review'].str.split('').explode().value_counts().to_frame().reset_index()
char_count.columns = ['char', 'count']
char_count

,char,count
0,,32505245
1,e,17192173
2,t,11887238
3,a,11535089
4,o,10078141
...,...,...
90,\,115
91,},87
92,{,82
93,`,25


In [59]:
# Get the slice of the less frequent, let's inspect
char_count.iloc[65:]

,char,count
65,U,45309
66,4,45284
67,J,38386
68,K,37575
69,&,32803
70,6,23887
71,9,23570
72,8,22471
73,7,21492
74,;,17783


In [62]:
# Remove some of these, rest seem plausible
df['review'] = df['review'].apply(lambda x: re.sub(r'[{}*\r\\\^`|]', '', x))
df['review']

0         Overall the food is pretty good but portions c...
1         We've been to ani ramen in Montclair New Jerse...
2         The real deal Japanese rice. Crispy tofu was t...
3         This is located within Brookfield place. You c...
4         I have been wanting to try Ani Ramen for the l...
                                ...                        
331688    I honestly think between 161-170th in the area...
331689    Awesome pizza for the area.  Prices can be a l...
331690    I called to order. When I gave my address, whi...
331691    I called this place to place a delivery order....
331692    Nonna's is quality pizza, and not just for da ...
Name: review, Length: 331501, dtype: object

In [63]:
set(' '.join(df['review'])) # Now looks cleaner, ready for VADER

{' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '~'}

In [ ]:
# Now start the analysis